In [39]:
import sys
import os
import json
from google.protobuf.json_format import Parse
sys.path.insert(0, 'compiled_protobufs')
from taskmap_pb2 import TaskMap
from pyserini.search import LuceneSearcher
import csv

In [7]:
searcher_config = {
    'BM25': {'b': 0.4, 'k1': 0.9},
    # 'BM25+RM3':{
    #     'BM25': {'b': 0.4, 'k1': 0.9},
    #     'RM3': {'fb_terms': 10, 'fb_docs': 10, 'original_query_weight': 0.5},
    # }
}

In [40]:
taskmap_cooking_index_path = os.path.join(os.getcwd(), "indexes", "food", "system_index_sparse")
annotations_path = os.path.join(os.getcwd(), "datasets", "judgments", "cooking-annotations.csv")

def get_pyserini_docs(searcher, query, k, query_id):
    """ Print pyserini results based on specific query, searcher, and k. """
    results = []
    for hit in searcher.search(q=query, k=k):
        doc_string = hit.raw
        doc_json = json.loads(doc_string)
        taskmap_json = doc_json['recipe_document_json']
        taskmap = Parse(json.dumps(taskmap_json), TaskMap())
        result = {
            "doc-id" : taskmap.taskmap_id, 
            "doc-title" : taskmap.title, 
            "doc-url" : taskmap.source_url, 
            "score": round(float(hit.score),3),
            "query-id": query_id,
            "query": query
        }
        results.append(result)
    return results

In [42]:
query = "Roast Tenderloin of Beef"
searcher = LuceneSearcher(index_dir=taskmap_cooking_index_path)
searcher.set_bm25(b=0.4,k1=0.9)

new_judgments = []
for idx, query in cooking_queries.iterrows():
    print(query["target query"])
    new_judgments += get_pyserini_docs(searcher, query["target query"], k=5, query_id=idx)
    

with open(annotations_path, 'w') as csvfile:
    fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=new_judgments[0].keys())
    writer.writeheader()
    writer.writerows(new_judgments)


risotto without mushrooms
salty vegan cheesecake
fluffy chocolate cake famous cook
pumpkin soup without blender
 Robuchon tomato soup
lime lemonade without lemons
 uncle roger egg fried rice
pescatarian friendly meatballs
light greek dinner
pre workout banana smoothie
